In [8]:
!pip install -q transformers


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
from google.colab import drive

drive.mount('/content/drive')

%cd ./drive/MyDrive/Grounding_lm

Mounted at /content/drive


# Data

In [1]:
import pandas as pd
import numpy as np

In [7]:
df_wiki_dev = pd.read_json('data/QA-dataset/data/2wikimultihopQA/dev.json')

block_records = np.load('data/block_records.npy', allow_pickle=True)

# Pre-Hoc

## REALM

In [11]:
import torch
from transformers import AutoTokenizer, RealmForOpenQA, RealmRetriever

tokenizer = AutoTokenizer.from_pretrained("google/realm-orqa-nq-openqa")
retriever = RealmRetriever(block_records=block_records, tokenizer=tokenizer)
model = RealmForOpenQA.from_pretrained("google/realm-orqa-nq-openqa", retriever=retriever)

RuntimeError: [enforce fail at ..\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 6837103616 bytes.

In [ ]:
question = df_wiki_dev['question'][0]
question_ids = tokenizer([question], return_tensors="pt")
answer_ids = tokenizer(
    [df_wiki_dev['answer'][0]],
    add_special_tokens=False,
    return_token_type_ids=False,
    return_attention_mask=False,
).input_ids

reader_output, predicted_answer_ids = model(**question_ids, answer_ids=answer_ids, return_dict=False)
predicted_answer = tokenizer.decode(predicted_answer_ids)
loss = reader_output.loss
